In [1]:
import geopandas as gpd
import requests
from pathlib import Path
import folium
from folium import GeoJson

from shapely.geometry import Point

OUTPUT_DIR = Path('generated')
OUTPUT_DIR.mkdir(exist_ok=True)

# via Google Maps
COOP_RACINE_GEOCODE = Point(-87.65413, 41.76519)

TRACT_SHAPEFILE_PATH = Path('tl_2024_17_tract.zip')
TRACT_SHAPEFILE_URL = 'https://www2.census.gov/geo/tiger/TIGER2024/TRACT/tl_2024_17_tract.zip'

if not TRACT_SHAPEFILE_PATH.exists():
    response = requests.get(TRACT_SHAPEFILE_URL)
    TRACT_SHAPEFILE_PATH.write_bytes(response.content)


In [ ]:
# Read the shapefile into a GeoDataFrame
tracts_gdf = gpd.read_file(f"zip://{TRACT_SHAPEFILE_PATH}")

# Ensure CRS is set to EPSG:4326 (WGS84)
tracts_gdf = tracts_gdf.to_crs(epsg=4326)

# Filter to only Cook County (COUNTYFP 031)
cook_county_tracts = tracts_gdf[tracts_gdf['COUNTYFP'] == '031'].copy()

# Convert COOP_RACINE_GEOCODE to a GeoDataFrame with the same CRS
coop_point_gdf = gpd.GeoDataFrame({'geometry': [COOP_RACINE_GEOCODE]}, crs="EPSG:4326")

# Use NAD83 UTM Zone 16N (EPSG:26916) for accurate distance measurements in Chicago
cook_county_tracts_proj = cook_county_tracts.to_crs(epsg=26916)  # UTM Zone 16N
coop_point_proj = coop_point_gdf.to_crs(epsg=26916)

# Buffer the point by 1 mile (1609.34 meters)
buffer_1mile_proj = coop_point_proj.geometry.buffer(1609.34).iloc[0]

# Select tracts that intersect with the 1 mile buffer
tracts_within_1mile_proj = cook_county_tracts_proj[cook_county_tracts_proj.geometry.intersects(buffer_1mile_proj)].copy()

# Convert buffer back to WGS84 for display
buffer_1mile_wgs84 = gpd.GeoSeries([buffer_1mile_proj], crs=26916).to_crs(epsg=4326).iloc[0]

# Convert selected tracts back to WGS84
tracts_within_1mile = tracts_within_1mile_proj.to_crs(epsg=4326)

# Calculate the percentage of each tract's area that falls within the 1-mile buffer
tracts_within_1mile['pct_in_circle'] = 0.0

for idx, row in tracts_within_1mile.iterrows():
    # Get the tract geometry in projected CRS
    tract_geom_proj = cook_county_tracts_proj[cook_county_tracts_proj['GEOID'] == row['GEOID']].geometry.iloc[0]
    
    # Calculate total area of tract
    total_area = tract_geom_proj.area
    
    # Calculate intersection area with buffer
    intersection = tract_geom_proj.intersection(buffer_1mile_proj)
    intersection_area = intersection.area
    
    # Calculate percentage
    pct = (intersection_area / total_area) * 100 if total_area > 0 else 0
    tracts_within_1mile.loc[idx, 'pct_in_circle'] = pct

# Sort by GEOID for consistent output
tracts_within_1mile = tracts_within_1mile.sort_values('GEOID')

# Debug output
print(f"\nProjection: NAD83 UTM Zone 16N (EPSG:26916)")
print(f"Total Cook County tracts: {len(cook_county_tracts)}")
print(f"Tracts within 1 mile: {len(tracts_within_1mile)}")
print(f"\nSelected tract GEOIDs with coverage percentages:")
for _, row in tracts_within_1mile.iterrows():
    print(f"  {row['GEOID']}: {row['pct_in_circle']:.1f}%")

In [3]:
# Create a folium map centered at the COOP_RACINE_GEOCODE location
m = folium.Map(
    location=[COOP_RACINE_GEOCODE.y, COOP_RACINE_GEOCODE.x], 
    zoom_start=14,
    tiles='OpenStreetMap'
)

# Add the highlighted tracts within 1 mile
for idx, row in tracts_within_1mile.iterrows():
    tract_id = row['GEOID']
    # Get percentage if it exists, otherwise default to 0
    pct_in_circle = row.get('pct_in_circle', 0)
    census_reporter_url = f"https://censusreporter.org/profiles/14000US{tract_id}"
    
    popup_html = f'''<a href="{census_reporter_url}" target="_blank">
    <b>Tract {tract_id}</b><br>
    {pct_in_circle:.1f}% within 1-mile circle<br>
    Click to view on Census Reporter
    </a>'''
    
    folium.GeoJson(
        row.geometry,
        style_function=lambda x: {
            'fillColor': 'lightblue',
            'color': 'darkblue',
            'weight': 2,
            'fillOpacity': 0.6
        },
        tooltip=f"Tract: {tract_id} ({pct_in_circle:.1f}% in circle)",
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(m)
    
    # Add label at the centroid
    centroid = row.geometry.centroid
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.DivIcon(html=f'<div style="font-size: 10pt; color: black; font-weight: bold; text-shadow: -1px -1px 0 white, 1px -1px 0 white, -1px 1px 0 white, 1px 1px 0 white;">{tract_id}</div>')
    ).add_to(m)

# Add the ACTUAL buffer polygon that was used for selection
# This is converted from projected CRS, so it may look slightly distorted
# Make it non-interactive with pointer-events: none
folium.GeoJson(
    buffer_1mile_wgs84,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.05,
        'dashArray': '5, 5'
    },
    name='Selection buffer (converted from UTM 16N)',
    popup='This is the actual buffer used for tract selection',
    overlay=True,
    control=True,
    show=True,
    smooth_factor=0
).add_to(m)

# Add the 1-mile circle using folium.Circle for comparison
# Make it non-interactive with pointer-events: none
folium.Circle(
    location=[COOP_RACINE_GEOCODE.y, COOP_RACINE_GEOCODE.x],
    radius=1609.34,  # radius in meters
    color='blue',
    fill=True,
    fillColor='blue',
    fillOpacity=0.1,
    weight=2,
    popup='1-mile radius (folium.Circle)'
).add_to(m)

# Add a marker at the center point
folium.Marker(
    location=[COOP_RACINE_GEOCODE.y, COOP_RACINE_GEOCODE.x],
    popup='Cooperation Racine',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Add custom CSS to make circles click-through
# This allows clicks to pass through to tracts underneath
click_through_css = """
<style>
path.leaflet-interactive[stroke='blue'],
path.leaflet-interactive[stroke='red'] {
    pointer-events: none !important;
}
</style>
"""
m.get_root().html.add_child(folium.Element(click_through_css))

# Add layer control
folium.LayerControl().add_to(m)

print(f"Map created with {len(tracts_within_1mile)} highlighted tracts")
print("Red dashed line = actual selection buffer (from UTM 16N)")
print("Blue solid line = folium.Circle for comparison")
print("Circles are click-through - you can click tracts underneath")

# Display the map
m

Map created with 24 highlighted tracts
Red dashed line = actual selection buffer (from UTM 16N)
Blue solid line = folium.Circle for comparison
Circles are click-through - you can click tracts underneath


In [ ]:
# Save the tract data with percentages to files

# Display summary table
print("Tracts within 1-mile radius with percentage coverage:")
print(tracts_within_1mile[['GEOID', 'NAME', 'pct_in_circle']].to_string(index=False))
print(f"\nTotal tracts: {len(tracts_within_1mile)}")

# Save as CSV (without geometry and some other noisy stuff)
csv_data = tracts_within_1mile.drop(columns=['geometry', 'STATEFP', 'COUNTYFP', 
    'NAME', 'TRACTCE', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON',])
output_csv = f'{OUTPUT_DIR}/tracts_within_1mile.csv'
csv_data.to_csv(output_csv, index=False)
print(f"\nSaved CSV to: {output_csv}")

# Save as GeoJSON (with geometry)
output_json = f'{OUTPUT_DIR}/tracts_within_1mile.geojson'
tracts_within_1mile.to_file(output_json, driver='GeoJSON')
print(f"Saved GeoJSON to: {output_json}")

In [5]:
# Verify the buffer size and save the map
from shapely.geometry import Point as ShapelyPoint

# Get the bounds of the buffer in the projected CRS
buffer_bounds = buffer_1mile_proj.bounds
print(f"Buffer bounds (EPSG:3857): {buffer_bounds}")

# Calculate the approximate radius
center_point = coop_point_proj.geometry.iloc[0]
print(f"Center point (EPSG:3857): {center_point.x}, {center_point.y}")

# Calculate distance from center to edge
radius_meters = center_point.distance(ShapelyPoint(buffer_bounds[2], center_point.y))
print(f"Calculated radius: {radius_meters:.2f} meters ({radius_meters / 1609.34:.2f} miles)")
print(f"Expected: 1609.34 meters (1.00 miles)")

# Save the map as an HTML file that you can open in a browser
output_html = f"{OUTPUT_DIR}/chicago_tracts_map.html"
m.save(output_html)
print(f"\nMap saved to {output_html} - open this file in your browser for full interactivity")

Buffer bounds (EPSG:3857): (444018.3305142864, 4622303.47480407, 447237.01051428646, 4625522.15480407)
Center point (EPSG:3857): 445627.67051428644, 4623912.81480407
Calculated radius: 1609.34 meters (1.00 miles)
Expected: 1609.34 meters (1.00 miles)

Map saved to generated/chicago_tracts_map.html - open this file in your browser for full interactivity


In [6]:
# Diagnostic: Check what percentage of the circle is covered by tracts
from shapely.ops import unary_union

# Get the union of all selected tracts in projected CRS
all_tracts_union = unary_union(tracts_within_1mile_proj.geometry)

# Calculate intersection with buffer
covered_area = all_tracts_union.intersection(buffer_1mile_proj)
buffer_area = buffer_1mile_proj.area

coverage_pct = (covered_area.area / buffer_area) * 100

print(f"Circle buffer area: {buffer_area:,.2f} sq meters")
print(f"Area covered by tracts: {covered_area.area:,.2f} sq meters")
print(f"Coverage: {coverage_pct:.2f}%")

# Calculate the uncovered area
if coverage_pct < 100:
    uncovered = buffer_1mile_proj.difference(all_tracts_union)
    print(f"\nUncovered area: {uncovered.area:,.2f} sq meters ({100-coverage_pct:.2f}% of circle)")

Circle buffer area: 8,123,582.92 sq meters
Area covered by tracts: 8,123,582.92 sq meters
Coverage: 100.00%

Uncovered area: 0.00 sq meters (0.00% of circle)
